<a href="https://colab.research.google.com/github/rer10013/ML-project/blob/main/Pytorch_Basic_3_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# libraries
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Mnist dataset
!git clone https://github.com/baek2sm/ml.git
!tar -zxvf ./ml/datasets/MNIST.tar.gz

# import datasets
path = './'
train_dataset = datasets.MNIST(path, train=True, download=True)
test_dataset = datasets.MNIST(path, train=False, download=True)

Cloning into 'ml'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 299 (delta 6), reused 3 (delta 3), pack-reused 289 (from 1)
Receiving objects: 100% (299/299), 64.38 MiB | 42.56 MiB/s, done.
Resolving deltas: 100% (155/155), done.
MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/train-images-idx3-ubyte.gz
MNIST/processed/
MNIST/processed/training.pt
MNIST/processed/test.pt


In [3]:
# train set
X_train, y_train = train_dataset.data / 255, train_dataset.targets
# test set
X_test, y_test = test_dataset.data / 255, test_dataset.targets

In [4]:
# check data
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

X_train:  torch.Size([60000, 28, 28])
y_train:  torch.Size([60000])
X_test:  torch.Size([10000, 28, 28])
y_test:  torch.Size([10000])


In [5]:
# 2d to 3d
X_train, X_test = X_train.unsqueeze(1), X_test.unsqueeze(1)

# check data
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  torch.Size([60000, 1, 28, 28])
X_test:  torch.Size([10000, 1, 28, 28])


In [6]:
# make datasheet
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# make batch dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
# CNN model class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0,5)
        )

        self.hidden_layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0,5)
        )

        self.hidden_layer3 = nn.Linear(128*5*5, 128)
        self.output_layer = nn.Linear(128, 10)

    def forward(self, x):
        out = self.hidden_layer1(x)
        out = self.hidden_layer2(out)
        out = out.view(out.size(0), -1)
        out = self.hidden_layer3(out)
        out = self.output_layer(out)
        return out

In [8]:
# check model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# make CNN object
model = CNN().to(device)

# make CE loss model object
criterion = nn.CrossEntropyLoss()

# make Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
# define train model
def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    # batch train
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        y_predicted = torch.argmax(hypothesis, dim=1)
        acc = (y_predicted == y_batch).float().mean()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

In [10]:
# define evaluation model
def evaluate(model, criterion, loader):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            y_predicted = torch.argmax(hypothesis, dim=1)
            acc = (y_predicted == y_batch).float().mean()
            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
# train model
num_epochs = 20
previous_train_loss, previous_train_acc = 1, 0
previous_test_loss, previous_test_acc = 1, 0
for epoch in range(1, num_epochs+1):
    current_train_loss, current_train_acc = train(model, criterion, optimizer, train_loader)
    current_test_loss, current_test_acc = evaluate(model, criterion, test_loader)

    print(f'epoch: {epoch}, train_loss: {current_train_loss:.4f}, train_acc: {current_train_acc:.4f}, test_loss: {current_test_loss:.4f}, test_acc: {current_test_acc:.4f} | {"boolshit" if current_train_acc + current_test_acc < previous_train_acc + previous_test_acc else "awesome"}')
    previous_train_loss, previous_train_acc = current_train_loss, current_train_acc
    previous_test_loss, previous_test_acc = current_test_loss, current_test_acc